# AutoGuessQuery 测试

本笔记本用于测试 `src/autocoder/agent/auto_guess_query.py` 中的 AutoGuessQuery 类功能。

In [1]:
# 初始化环境
import os
import sys
from autocoder.agent.auto_guess_query import AutoGuessQuery, NextQuery
import byzerllm

In [2]:
# 初始化 LLM 和 AutoGuessQuery
llm = byzerllm.ByzerLLM.from_default_model(model="deepseek_chat")
project_dir = "/Users/allwefantasy/projects/auto-coder"
auto_guesser = AutoGuessQuery(llm, project_dir,skip_diff=True)

2024-12-27 10:06:49.098 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-12-27 10:06:49,158	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-12-27 10:06:49,172	INFO worker.py:1740 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


In [3]:
# 测试 parse_history_tasks 方法
history_tasks = auto_guesser.parse_history_tasks()
print(f"Parsed {len(history_tasks)} history tasks")
for task in history_tasks:
    print(f"Query: {task[0]}")
    print(f"Files: {task[1]}")
    print(f"Diff: {task[2][:100]}..." if task[2] else "No diff")
    print("-" * 40)

Parsed 100 history tasks
Query: @@predict_next_task(location: src/autocoder/agent/auto_guess_query.py) 改成 predict_next_tasks 也就是返回一组可选 NextQuery,可能性由高到低排列. 需要同步修改 @@guess_next_query(location: src/autocoder/agent/auto_guess_query.py) 让他返回json 数组,同时给定示例
Files: ['/Users/allwefantasy/projects/auto-coder/src/autocoder/agent/auto_filegroup.py', '/Users/allwefantasy/projects/auto-coder/src/autocoder/agent/auto_guess_query.py', '.auto-coder/libs/llm_friendly_packages/github.com/allwefantasy/byzer-llm/README.md']
No diff
----------------------------------------
Query: 参考 @test_auto_filegroup.ipynb 生成对 @src/autocoder/agent/auto_guess_query.py的 jupyter notebook
Files: ['/Users/allwefantasy/projects/auto-coder/src/autocoder/agent/auto_filegroup.py', '/Users/allwefantasy/projects/auto-coder/src/autocoder/agent/auto_guess_query.py', '.auto-coder/libs/llm_friendly_packages/github.com/allwefantasy/byzer-llm/README.md']
No diff
----------------------------------------
Query: 参考 @src/autocoder/agent/aut

In [5]:
# 测试 predict_next_task 方法
next_tasks = auto_guesser.predict_next_tasks()
for next_task in next_tasks:
    print(f"Next Query: {next_task.query}")
    print(f"Predicted Files: {next_task.urls}")
    print(f"Priority: {next_task.priority}")
    print(f"Reason: {next_task.reason}")
    print(f"Dependencies: {next_task.dependency_queries}")
    print("-" * 40)

Next Query: 在 auto_guess_query.py 中添加任务预测的单元测试
Predicted Files: ['/Users/allwefantasy/projects/auto-coder/src/autocoder/agent/auto_guess_query.py', '/Users/allwefantasy/projects/auto-coder/notebooks/tests/test_auto_guess_query.ipynb']
Priority: 5
Reason: 任务预测功能是核心功能，需要确保其正确性和稳定性。添加单元测试可以验证预测逻辑的准确性，防止回归问题。
Dependencies: ['改成 predict_next_tasks 也就是返回一组可选 NextQuery,可能性由高到低排列', '参考 @test_auto_filegroup.ipynb 生成对 @src/autocoder/agent/auto_guess_query.py的 jupyter notebook']
----------------------------------------
Next Query: 优化 auto_filegroup.py 中的分组性能，支持大文件处理
Predicted Files: ['/Users/allwefantasy/projects/auto-coder/src/autocoder/agent/auto_filegroup.py', '/Users/allwefantasy/projects/auto-coder/src/autocoder/utils/__init__.py']
Priority: 4
Reason: 当前分组逻辑在处理大文件时可能存在性能问题，需要优化文件处理逻辑，增加对大文件的支持，提升整体性能。
Dependencies: ['优化下 @@group_by_similarity(location: src/autocoder/agent/auto_filegroup.py)', '重构 @src/autocoder/agent/auto_filegroup.py 的代码']
----------------------------------------
Next Query

In [ ]:
# 测试 NextQuery 模型
sample_next_query = NextQuery(
    query="Implement new feature X",
    urls=["src/feature_x.py", "tests/test_feature_x.py"],
    priority=3,
    reason="Required for upcoming release",
    dependency_queries=["Setup basic project structure"]
)
print(sample_next_query.json(indent=2))

## 测试说明

1. `parse_history_tasks` 测试：验证是否能正确解析历史任务
2. `predict_next_task` 测试：验证是否能正确预测下一个任务
3. `NextQuery` 模型测试：验证数据模型是否能正常工作

注意：测试前请确保在 `tests/resources/test_project/actions` 目录下存在有效的 YAML 文件。